# Capstone Project - The Battle of the Neighborhoods (Final Assignment)

*This notebook is mainly for Applied Data Science Capstone - Coursera Course, and should not be used for references or clinical judgement*

## Table of contents
- [Introduction](#introduction)
- [Data](#data)
- [Methodology and Analysis](#methodology-and-analysis)
- [Results](#results)
- [Discussion](#discussion)
- [Conclusion](#conclusion)


## Introduction

Coronavirus disease (COVID-19) is an infectious disease caused by a newly discovered coronavirus.  
In Thailand, the city which has most cases of COVID-19 is Bangkok - the capital and most populous city of Thailand.
The city divided into 50 districts in total.  
In this project we will try to find the correlation of number of COVID-19 cases who had onset in each district of Bangkok and the clusters of districts based on catogery and density of venues

This data and visualization may provided useful information for strategy to deal with current COVID-19 spreads.
For example may promotes the awareness of people, or lead to invented policies to limited the spread of the disease.

## Data

The data we need are list below  

- Number of COVID-19 cases who had onset in each district of Bangkok - this data is query and process from https://covid19.th-stat.com/api/open/cases - official API provided by Department of Disease Control of Thailand
- Coordinates data of each district of Bangkok - together with Foursquare API to find the venues and create clusters of district. I used data from https://data.go.th/dataset/item_c6d42e1b-3219-47e1-b6b7-dfe914f27910 - Open Government Data of Thailand. Which included the data of latitude and longitude of each subdistrict in Thailand
- Table included lists of district in Bangkok included both English and Thai name of each district from https://en.wikipedia.org/wiki/List_of_districts_of_Bangkok
- GeoJSON file of districts in Bangkok from https://github.com/pcrete/gsvloader-demo/blob/master/geojson/Bangkok-districts.geojson for visualize chloropleth map

### COVID-19 cases

In [16]:
import requests
import pandas as pd

url = 'https://covid19.th-stat.com/api/open/cases'
response = requests.get(url)
covid = pd.json_normalize(response.json()["Data"])
print(covid.shape)
covid.head()

(10081, 13)


,ConfirmDate,No,Age,Gender,GenderEn,Nation,NationEn,Province,ProvinceId,District,ProvinceEn,Detail,StatQuarantine
0,2021-01-09 15:31:58,10053,41.0,หญิง,Female,Thailand,Thailand,จันทบุรี,7,โป่งน้ำร้อน,Chanthaburi,None,1
1,2021-01-09 15:31:58,10052,41.0,ชาย,Male,Thailand,Thailand,จันทบุรี,7,โป่งน้ำร้อน,Chanthaburi,None,1
2,2021-01-09 15:31:58,10051,32.0,หญิง,Female,Thailand,Thailand,จันทบุรี,7,โป่งน้ำร้อน,Chanthaburi,None,1
3,2021-01-09 15:31:58,10050,33.0,หญิง,Female,Thailand,Thailand,ระยอง,50,เมืองระยอง,Rayong,None,1
4,2021-01-09 15:31:58,10049,68.0,ชาย,Male,Thailand,Thailand,ระยอง,50,เมืองระยอง,Rayong,None,1


In [14]:
covid_bkk = covid[covid['ProvinceId']==1]
print(covid_bkk.shape)
covid_bkk.head()

(2716, 13)


,ConfirmDate,No,Age,Gender,GenderEn,Nation,NationEn,Province,ProvinceId,District,ProvinceEn,Detail,StatQuarantine
65,2021-01-09 15:31:57,9976,48.0,ชาย,Male,Thailand,Thailand,กรุงเทพมหานคร,1,บางขุนเทียน,Bangkok,None,1
66,2021-01-09 15:31:57,9975,30.0,หญิง,Female,Thailand,Thailand,กรุงเทพมหานคร,1,บางขุนเทียน,Bangkok,None,1
67,2021-01-09 15:31:57,9974,27.0,ชาย,Male,Thailand,Thailand,กรุงเทพมหานคร,1,วังทองหลาง,Bangkok,None,1
68,2021-01-09 15:31:57,9973,50.0,หญิง,Female,Thailand,Thailand,กรุงเทพมหานคร,1,บางขุนเทียน,Bangkok,None,1
69,2021-01-09 15:31:57,9972,48.0,ชาย,Male,Thailand,Thailand,กรุงเทพมหานคร,1,บางขุนเทียน,Bangkok,None,1


In [37]:
print(covid_bkk['District'].unique().size)
covid_bkk['District'].unique()

150


array(['บางขุนเทียน', 'วังทองหลาง', 'วัฒนา', 'เมือง', 'พระนคร', 'ธนบุรี',
       'สวนหลวง', 'บางบอน', 'บางแค', 'ราษฎร์บูรณะ', 'พระสมุทรเจดีย์',
       'ประเวศ', 'พระประแดง', 'บางนา', 'ภาษีเจริญ', 'ท่าวุ้ง', 'บางกะปิ',
       'เมืองสมุทรสาคร', 'บางเขน', 'บางพลัด', 'บางคอแหลม', 'บางพลี',
       'บางละมุง', 'บางกรวย', 'อัมพวา', 'เสนา', 'บางบัวทอง', 'ดอนเมือง',
       'บางบ่อ', 'คลองเตย', 'คลองสาน', 'บางซื่อ', 'คลองสามวา', 'บางรัก',
       'บ้านกุ่ม', 'จอมทอง', 'หนองแขม', 'ปทุมวัน', 'บางใหญ่', 'บางเสาธง',
       'ยานนาวา', 'บางกอกใหญ่', 'ปากเกร็ด', 'ศรีขระภูมิ', 'จตุจักร',
       'มีนบุรี', 'ดุสิต', 'จอมบึง', 'พญาไท', 'ทวีวัฒนา', '', 'สาทร',
       'ห้วยขวาง', 'กระทุ่มแบน', 'วััฒนา', 'บางพลีัด',
       'ป้อมปราการศัตรูพ่าย', 'พระโขนง', 'ดินแดง', 'บางนอน', 'ลาดกระบัง',
       'ทุ่งครุ', 'หลักสี่', 'รัชดา', 'บางขุุนเทียน', 'บึงกุ่ม', '้เมือง',
       'ยายนาวา', 'หนองจอก', 'บางกอกน้อย', 'แก้งคร้อ', 'บ้านโป่ง', 'สาธร',
       'คลองหลวง', 'ป้อมปราบศัตรูพ่าย', 'ราชเทวี', 'บางมด', 'บางยี่ขัน',
  

Note that there is only 50 district in Bangkok. This list we got include some typo data and etc.  
We will get the list of all 50 districts, then match with the data we got earlier, and drop all the data that not matched

First, get the name of 50 districts in Bangkok from wikipedia webpage

In [23]:
bkk_district = pd.read_html('https://en.wikipedia.org/wiki/List_of_districts_of_Bangkok')[0]
bkk_district.head()

,District(Khet),MapNr,Post-code,Thai,Popu-lation,No. ofSubdis-trictsKhwaeng,Latitude,Longitude
0,Bang Bon,50,10150,บางบอน,105161,4,13.659200,100.399100
1,Bang Kapi,6,10240,บางกะปิ,148465,2,13.765833,100.647778
2,Bang Khae,40,10160,บางแค,191781,4,13.696111,100.409444
3,Bang Khen,5,10220,บางเขน,189539,2,13.873889,100.596389
4,Bang Kho Laem,31,10120,บางคอแหลม,94956,3,13.693333,100.502500


In [41]:
bkk_district_list = bkk_district['Thai'].to_list()
print("Total: ",len(bkk_district_list))
print("The first ten samples below")
bkk_district_list[0:10]

Total:  50
The first ten samples below


['บางบอน',
 'บางกะปิ',
 'บางแค',
 'บางเขน',
 'บางคอแหลม',
 'บางขุนเทียน',
 'บางนา',
 'บางพลัด',
 'บางรัก',
 'บางซื่อ']

### Coordinates data

### Foursquare

## Methodology and Analysis

## Results

## Discussion

## Conclusion